In [1]:
import keras
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense, Activation

Using Theano backend.


In [2]:
import numpy as np
from multigraph import MultiGraph

In [3]:
#import csv
import unicodecsv as csv
""" EXAMPLE
add,IS,operator
subtract,IS,operator
multiply,IS,operator
divide,IS,operator
open_closure,IS,operator
close_closure,IS,operator
"""
op_graph = MultiGraph()
#with open('operator_graph.csv', 'r') as csvfile:
for i in range(45):
    print('loading relation '+str(i)+'...')
    with open('aifb_csv/aifb_relation_'+str(i)+'.csv', 'r') as csvfile:
        graphreader = csv.reader(csvfile, delimiter=",")
        for row in graphreader:
            #print(row)
            op_graph.add_connection(row)

loading relation 0...
loading relation 1...
loading relation 2...
loading relation 3...
loading relation 4...
loading relation 5...
loading relation 6...
loading relation 7...
loading relation 8...
loading relation 9...
loading relation 10...
loading relation 11...
loading relation 12...
loading relation 13...
loading relation 14...
loading relation 15...
loading relation 16...
loading relation 17...
loading relation 18...
loading relation 19...
loading relation 20...
loading relation 21...
loading relation 22...
loading relation 23...
loading relation 24...
loading relation 25...
loading relation 26...
loading relation 27...
loading relation 28...
loading relation 29...
loading relation 30...
loading relation 31...
loading relation 32...
loading relation 33...
loading relation 34...
loading relation 35...
loading relation 36...
loading relation 37...
loading relation 38...
loading relation 39...
loading relation 40...
loading relation 41...
loading relation 42...
loading relation 43..

In [4]:
a_graph = op_graph.get_adjacency_matrix()
print("number of nodes",op_graph.n_nodes)
print("number of relations",op_graph.n_rels)
print("relations:")
for k,v in op_graph.rel_counter.iteritems():
    print(k.rsplit('/',1)[-1], v)

('number of nodes', 8284)
('number of relations', 45)
relations:
(u'ontology#number', 145)
(u'ontology#isWorkedOnBy', 571)
(u'ontology#worksAtProject', 200)
(u'owl#allValuesFrom', 152)
(u'ontology#dealtWithIn', 357)
(u'owl#onProperty', 152)
(u'ontology#type', 50)
(u'ontology#author', 3986)
(u'ontology#abstract', 534)
(u'ontology#carriedOutBy', 79)
(u'ontology#month', 759)
(u'ontology#phone', 227)
(u'22-rdf-syntax-ns#type', 4124)
(u'ontology#address', 202)
(u'ontology#note', 114)
(u'ontology#publication', 4163)
(u'ontology#financedBy', 65)
(u'ontology#chapter', 15)
(u'ontology#editor', 190)
(u'ontology#pages', 548)
(u'owl#inverseOf', 10)
(u'ontology#projectInfo', 952)
(u'type', 129)
(u'ontology#edition', 12)
(u'ontology#booktitle', 765)
(u'ontology#isAbout', 2477)
(u'ontology#finances', 68)
(u'ontology#howpublished', 49)
(u'ontology#member', 339)
(u'ontology#hasProject', 952)
(u'ontology#isbn', 16)
(u'ontology#journal', 161)
(u'ontology#year', 1227)
(u'ontology#title', 1227)
(u'ontology

In [5]:
x_train = a_graph
print('input dims:',x_train.shape)
print(type(x_train))
sum(list(x_train[5]))

('input dims:', (8284, 372780))
<class 'scipy.sparse.csr.csr_matrix'>


<1x372780 sparse matrix of type '<type 'numpy.float32'>'
	with 18 stored elements in Compressed Sparse Row format>

In [37]:
# Need to make a custom Dense layer to tie weights between the encoder and decoder
from keras import backend as K

class DenseTied(Dense):
    def __init__(self, output_dim, master_layer, **kwargs):
        #output_dim needs to be equal to the input dimensions of the master_layer
        self.master_layer = master_layer
        self.output_dim = output_dim
        super(DenseTied, self).__init__(self.output_dim, **kwargs)
    
    def build(self,input_shape):
        self.kernel = K.transpose(self.master_layer.kernel)
        super(DenseTied, self).build(input_shape)
        
    #def call(self, inputs):
    #    output = K.dot(inputs, K.transpose(self.master_layer.kernel))
    #    output = K.bias_add(output, self.bias, data_format='channels_last')
    #    if self.activation is not None:
    #        output = self.activation(output)
    #    return output


In [38]:
encoding_dim = 128
input_dim = x_train.shape[1]

inputs = Input(shape=(input_dim,),name="inputs")
# Encoder Layers
encode_layer1 = Dense(4 * encoding_dim, activation='tanh',name="encode_layer1")
encode_layer2 = Dense(2 * encoding_dim, activation='tanh',name="encode_layer2")
coding_layer = Dense(encoding_dim, activation='tanh',name="coding_layer")

encoding_1 = encode_layer1(inputs)
encoding_2 = encode_layer2(encoding_1)
the_code = coding_layer(encoding_2)

# Decoder Layers
#decode_layer1 = Dense(2 * encoding_dim, activation='tanh',name="decode_layer1")
#decode_layer2 = Dense(4 * encoding_dim, activation='tanh',name="decode_layer2")
#reconstruction_layer = Dense(input_dim, activation='tanh',name="reconstruction_layer")
decode_layer1 = DenseTied(2 * encoding_dim, coding_layer, activation='tanh',name="decode_layer1")
decode_layer2 = DenseTied(4 * encoding_dim, encode_layer2, activation='tanh',name="decode_layer2")
recon_layer = DenseTied(input_dim, encode_layer1, activation='tanh',name="reconstruction_layer")

decoding_1 = decode_layer1(the_code)
decoding_2 = decode_layer2(decoding_1)
reconstruction = reconstruction_layer(decoding_2)

ae = Model(inputs=inputs, outputs=reconstruction)
#monitor = EarlyStopping(monitor='loss', min_delta=0.0001, patience=5, verbose=1, mode='auto')
ae.compile(optimizer='adam', loss='mse')
ae.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          (None, 372780)            0         
_________________________________________________________________
encode_layer1 (Dense)        (None, 512)               190863872 
_________________________________________________________________
encode_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
coding_layer (Dense)         (None, 128)               32896     
_________________________________________________________________
decode_layer1 (DenseTied)    (None, 256)               33024     
_________________________________________________________________
decode_layer2 (DenseTied)    (None, 512)               131584    
_________________________________________________________________
reconstruction_layer (Dense) (None, 372780)            191236140 
Total para

In [40]:
ae.fit(x_train, x_train.toarray(), epochs=2, verbose=1)

Epoch 1/2
 352/8284 [>.............................] - ETA: 16:36 - loss: 8.1241e-06

KeyboardInterrupt: 

In [ ]:
#coding_model = Model(inputs=ae.inputs, outputs=ae.get_layer("the_code").output)
coding_model = Model(inputs=inputs, outputs=the_code)

In [ ]:
#save the embeddings in order to plot them later
# serialize model to JSON
model_json = coding_model.to_json()
with open("coding_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
coding_model.save_weights("coding_model.h5")
print("Saved model to disk")

In [ ]:
import pickle
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

#save the adjacency matrix as well
save_object(op_graph, 'adj_graph.pkl')
print("Saved graph object to disk")